<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/langchain_neo4j_vertexai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade google-cloud-aiplatform langchain neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 12.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://34.239.133.123:7687",
    username="neo4j",
    password="traps-henry-milliliters"
)

In [2]:
from google.colab import files

uploaded = files.upload()

Saving credentials.json to credentials.json


In [3]:
import os

from langchain.chat_models import ChatVertexAI
from langchain.chains import GraphCypherQAChain

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "credentials.json"

In [4]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    ChatVertexAI(temperature=0, model_name="codechat-bison"), graph=graph, verbose=True,
)

chain.run("Who is the most followed fortnite streamer?")



> Entering new  chain...
Generated Cypher:

MATCH (s:Stream)-[:PLAYS]->(g:Game)
WHERE g.name = "Fortnite"
RETURN s.followers ORDER BY s.followers DESC LIMIT 1

Full Context:
[{'s.followers': 7269018}]

> Finished chain.


'The most followed Fortnite streamer is Ninja.'

In [6]:
chain.run("Please recommend a good chess stream?")



> Entering new  chain...
Generated Cypher:

MATCH (s:Stream)-[:PLAYS]->(g:Game)
WHERE g.name = "Chess"
RETURN s
ORDER BY s.total_view_count DESC
LIMIT 1

Full Context:
[{'s': {'followers': 8772, 'name': 'imsatranc', 'url': 'https://www.twitch.tv/imsatranc'}}]

> Finished chain.


'I recommend imsatranc. He has 8772 followers and his stream is at https://www.twitch.tv/imsatranc.'

In [14]:
chain.run("Which are the top five games played by streamers?")



> Entering new  chain...
Generated Cypher:

MATCH (s:Stream)-[:PLAYS]->(g:Game)
RETURN g.name AS game,
       COUNT(*) AS number_of_streams
ORDER BY number_of_streams DESC
LIMIT 5;

Full Context:
[{'game': 'Just Chatting', 'number_of_streams': 868}, {'game': 'Resident Evil Village', 'number_of_streams': 442}, {'game': 'Grand Theft Auto V', 'number_of_streams': 380}, {'game': 'League of Legends', 'number_of_streams': 279}, {'game': 'Fortnite', 'number_of_streams': 217}]

> Finished chain.


'The top five games played by streamers are:\n\n1. Just Chatting\n2. Resident Evil Village\n3. Grand Theft Auto V\n4. League of Legends\n5. Fortnite'

In [10]:
chain.run("Which streams are the most important based on PageRank?")



> Entering new  chain...
Generated Cypher:

MATCH (s:Stream)
WITH s AS stream
CALL pageRank(
    startNode: stream,
    maxIterations: 10,
    dampingFactor: 0.85
)
RETURN stream,
       rnk AS pageRank
ORDER BY rnk DESC
LIMIT 10



ValueError: ignored